In [2]:
import os
from flask import Flask, request, render_template, redirect, url_for
from werkzeug.utils import secure_filename
import numpy as np
import tensorflow as tf

ASSET_FOLDER = 'static/assets/'
UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__)
app.config['ASSET_FOLDER'] = ASSET_FOLDER
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

github_icon = os.path.join(app.config['ASSET_FOLDER'], 'github.png')
linkedin_icon = os.path.join(app.config['ASSET_FOLDER'], 'linkedin.png')
instagram_icon = os.path.join(app.config['ASSET_FOLDER'], 'instagram.png')
twitter_icon = os.path.join(app.config['ASSET_FOLDER'], 'twitter.png')
bg_video = os.path.join(app.config['ASSET_FOLDER'], 'pexels-tima-miroshnichenko-6010766.mp4')

# Load the model once at the start of the application
model_path = 'model.h5'
model = tf.keras.models.load_model(model_path)

@app.route('/')
def upload_form():
    return render_template('index.html', type="", githubicon=github_icon, linkedinicon=linkedin_icon, instagramicon=instagram_icon, twittericon=twitter_icon, bgvideo=bg_video)

@app.route('/', methods=['POST'])
def upload_image():
    try:
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)

        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Load and preprocess the image
        imvar = tf.keras.preprocessing.image.load_img(filepath, target_size=(176, 176))
        imarr = tf.keras.preprocessing.image.img_to_array(imvar)
        imarr = np.expand_dims(imarr, axis=0)  # Add batch dimension

        # Predict using the loaded model
        impred = model.predict(imarr)

        def roundoff(arr):
            arr[np.argwhere(arr != arr.max())] = 0
            arr[np.argwhere(arr == arr.max())] = 1
            return arr

        for i, classpreds in enumerate(impred):
            impred[i] = roundoff(classpreds)

        classcount = 1
        for count in range(4):
            if impred[0][count] == 1.0:
                break
            else:
                classcount += 1

        classdict = {1: "Mild Dementia", 2: "Moderate Dementia", 3: "No Dementia, Patient is Safe", 4: "Very Mild Dementia"}

        return render_template('index.html', type="Patient is suffering from " + classdict[classcount], githubicon=github_icon, linkedinicon=linkedin_icon, instagramicon=instagram_icon, twittericon=twitter_icon, bgvideo=bg_video)

    except Exception as e:
        return str(e), 500

if __name__ == "__main__":
    app.run(debug=False)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:45] "GET /static/assets/github.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:45] "GET /static/assets/linkedin.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:46] "GET /static/assets/twitter.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:46] "GET /static/assets/instagram.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:46] "GET /static/assets/pexels-tima-miroshnichenko-6010766.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 06:13:47]